In [1]:
import pandas as pd


def calculate_probabilities(data):
    # Calculate the probabilities of B
    p_b = data['B'].mean()

    # Calculate the probabilities of G given B
    p_g_given_b = data.groupby('B')['G'].mean()

    # Calculate the probabilities of F given G and C
    p_f_given_gc = data.groupby(['G', 'C'])['F'].mean()

    # Calculate the probabilities of C
    p_c = data['C'].mean()

    return p_b, p_g_given_b, p_f_given_gc, p_c
def read_training_data(file_name):
    data = []
    with open(file_name, 'r') as f:
        for line in f:
            data.append(list(map(int, line.strip().split())))
    return pd.DataFrame(data, columns=['B', 'G', 'C', 'F'])

def main():
    training_data_file = r"C:\Users\keert\OneDrive\Desktop\Ai3\ass2_k\ass2_k\training_data.txt"
    data = read_training_data(training_data_file)

    p_b, p_g_given_b, p_f_given_gc, p_c = calculate_probabilities(data)

    print(f"P(B=true) = {p_b}")
    print(f"P(G=true|B=true) = {p_g_given_b.loc[1]}")
    print(f"P(G=true|B=false) = {p_g_given_b.loc[0]}")
    print(f"P(F=true|G=true,C=true) = {p_f_given_gc.loc[1, 1]}")
    print(f"P(F=true|G=true,C=false) = {p_f_given_gc.loc[1, 0]}")
    print(f"P(F=true|G=false,C=true) = {p_f_given_gc.loc[0, 1]}")
    print(f"P(F=true|G=false,C=false) = {p_f_given_gc.loc[0, 0]}")
    print(f"P(C=true) = {p_c}")
if __name__ == '__main__':
    main()

P(B=true) = 0.3041095890410959
P(G=true|B=true) = 0.9279279279279279
P(G=true|B=false) = 0.11811023622047244
P(F=true|G=true,C=true) = 0.041666666666666664
P(F=true|G=true,C=false) = 0.7064220183486238
P(F=true|G=false,C=true) = 0.3157894736842105
P(F=true|G=false,C=false) = 0.9587628865979382
P(C=true) = 0.16986301369863013


In [3]:

import pandas as pd
import sys

def calculate_probabilities(data):
    # Calculate the probabilities of B
    p_b = data['B'].mean()

    # Calculate the probabilities of G given B
    p_g_given_b = data.groupby('B')['G'].mean()

    # Calculate the probabilities of F given G and C
    p_f_given_gc = data.groupby(['G', 'C'])['F'].mean()

    # Calculate the probabilities of C
    p_c = data['C'].mean()

    return p_b, p_g_given_b, p_f_given_gc, p_c

def read_training_data(file_name):
    data = []
    with open(file_name, 'r') as f:
        for line in f:
            data.append(list(map(int, line.strip().split())))
    return pd.DataFrame(data, columns=['B', 'G', 'C', 'F'])

def main():
    training_data_file = r"C:\Users\keert\OneDrive\Desktop\Ai3\ass2_k\ass2_k\training_data.txt"
    b_value = 'Bt'
    g_value = 'Gf'
    c_value = 'Ct'
    f_value = 'Ff'

    data = read_training_data(training_data_file)

    p_b, p_g_given_b, p_f_given_gc, p_c = calculate_probabilities(data)

    b = 1 if b_value == 'Bt' else 0
    g = 1 if g_value == 'Gt' else 0
    c = 1 if c_value == 'Ct' else 0
    f = 1 if f_value == 'Ft' else 0

    # Calculate P(B,G,C,F) using conditional probabilities
    p = (p_f_given_gc.loc[g,c] * p_g_given_b.loc[b] * p_c * p_b) / (p_g_given_b.loc[b] * p_c)

    print(f"P(B={b}, G={g}, C={c}, F={f}) = {p}")
    
main()


P(B=1, G=0, C=1, F=0) = 0.09603460706560923


In [2]:
import sys

import pandas as pd

# ... (previous functions remain unchanged)
def read_training_data(file_name):
    data = []
    with open(file_name, 'r') as f:
        for line in f:
            data.append(list(map(int, line.strip().split())))
    return pd.DataFrame(data, columns=['B', 'G', 'C', 'F'])

def parse_arguments(args):
    query_variables = {}
    evidence = {}
    given_flag = False
    for arg in args:
        if arg == "given":
            given_flag = True
        else:
            var, value = arg[0], arg[1]
            if given_flag:
                evidence[var] = value == "t"
            else:
                query_variables[var] = value == "t"
    return query_variables, evidence
def count_occurrences(data):
    counts = {}
    for _, row in data.iterrows():
        b = row['B']
        g = row['G']
        c = row['C']
        f = row['F']
        key = (b, g, c, f)
        counts[key] = counts.get(key, 0) + 1
    return counts

def joint_probability(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f):
    return p_b[b] * p_g_given_b[b][g] * p_f_given_gc[(g, c)][f] * p_c[c]
def calculate_probabilities(counts):
    total_count = sum(counts.values())

    p_b = {0: 0, 1: 0}
    p_g_given_b = {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}}
    p_f_given_gc = {(0, 0): {0: 0, 1: 0}, (0, 1): {0: 0, 1: 0}, (1, 0): {0: 0, 1: 0}, (1, 1): {0: 0, 1: 0}}
    p_c = {0: 0, 1: 0}

    for (b, g, c, f), count in counts.items():
        p_b[b] += count
        p_c[c] += count
        p_g_given_b[b][g] += count
        p_f_given_gc[(g, c)][f] += count

    p_b[0] /= total_count
    p_b[1] /= total_count
    p_c[0] /= total_count
    p_c[1] /= total_count

    for b in [0, 1]:
        total_g_given_b = sum(p_g_given_b[b].values())
        for g in [0, 1]:
            p_g_given_b[b][g] /= total_g_given_b

    for g, c in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        total_f_given_gc = sum(p_f_given_gc[(g, c)].values())
        for f in [0, 1]:
            p_f_given_gc[(g, c)][f] /= total_f_given_gc

    return p_b, p_g_given_b, p_f_given_gc, p_c
def calculate_jpd(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f):
    prob = p_b[b] * p_g_given_b[b][g] * p_f_given_gc[(g, c)][f] * p_c[c]
    return prob


def inference_by_enumeration(p_b, p_g_given_b, p_f_given_gc, p_c, query_variables, evidence):
    total_prob = 0
    for b in [0, 1]:
        for g in [0, 1]:
            for c in [0, 1]:
                for f in [0, 1]:
                    current_evidence = {'B': b, 'G': g, 'C': c, 'F': f}
                    if all(current_evidence[k] == v for k, v in evidence.items()):
                        prob = joint_probability(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f)
                        if all(current_evidence[k] == v for k, v in query_variables.items()):
                            total_prob += prob
    return total_prob

def main():
    training_data_file = r"C:\Users\keert\OneDrive\Desktop\Ai3\ass2_k\ass2_k\training_data.txt"
    data = read_training_data(training_data_file)
    counts = count_occurrences(data)
    p_b, p_g_given_b, p_f_given_gc, p_c = calculate_probabilities(counts)

    query_variables, evidence = parse_arguments(['Bt','Gf','Ff'])

    prob = inference_by_enumeration(p_b, p_g_given_b, p_f_given_gc, p_c, query_variables, evidence)
    print(f"P({query_variables} | {evidence}) = {prob}")


main()

import sys

import pandas as pd

# ... (previous functions remain unchanged)
def read_training_data(file_name):
    data = []
    with open(file_name, 'r') as f:
        for line in f:
            data.append(list(map(int, line.strip().split())))
    return pd.DataFrame(data, columns=['B', 'G', 'C', 'F'])

def parse_arguments(args):
    query_variables = {}
    evidence = {}
    given_flag = False
    for arg in args:
        if arg == "given":
            given_flag = True
        else:
            var, value = arg[0],arg[1]
            if given_flag:
                evidence[var] = value == "t"
            else:
                query_variables[var] = value == "t"
    return query_variables, evidence
def count_occurrences(data):
    counts = {}
    for _, row in data.iterrows():
        b = row['B']
        g = row['G']
        c = row['C']
        f = row['F']
        key = (b, g, c, f)
        counts[key] = counts.get(key, 0) + 1
    return counts

def joint_probability(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f):
    return p_b[b] * p_g_given_b[b][g] * p_f_given_gc[(g, c)][f] * p_c[c]
def calculate_probabilities(counts):
    total_count = sum(counts.values())

    p_b = {0: 0, 1: 0}
    p_g_given_b = {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}}
    p_f_given_gc = {(0, 0): {0: 0, 1: 0}, (0, 1): {0: 0, 1: 0}, (1, 0): {0: 0, 1: 0}, (1, 1): {0: 0, 1: 0}}
    p_c = {0: 0, 1: 0}

    for (b, g, c, f), count in counts.items():
        p_b[b] += count
        p_c[c] += count
        p_g_given_b[b][g] += count
        p_f_given_gc[(g, c)][f] += count

    p_b[0] /= total_count
    p_b[1] /= total_count
    p_c[0] /= total_count
    p_c[1] /= total_count

    for b in [0, 1]:
        total_g_given_b = sum(p_g_given_b[b].values())
        for g in [0, 1]:
            p_g_given_b[b][g] /= total_g_given_b

    for g, c in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        total_f_given_gc = sum(p_f_given_gc[(g, c)].values())
        for f in [0, 1]:
            p_f_given_gc[(g, c)][f] /= total_f_given_gc

    return p_b, p_g_given_b, p_f_given_gc, p_c
def calculate_jpd(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f):
    prob = p_b[b] * p_g_given_b[b][g] * p_f_given_gc[(g, c)][f] * p_c[c]
    return prob


def inference_by_enumeration(p_b, p_g_given_b, p_f_given_gc, p_c, query_variables, evidence):
    total_prob = 0
    for b in [0, 1]:
        for g in [0, 1]:
            for c in [0, 1]:
                for f in [0, 1]:
                    current_evidence = {'B': b, 'G': g, 'C': c, 'F': f}
                    if all(current_evidence[k] == v for k, v in evidence.items()):
                        prob = joint_probability(p_b, p_g_given_b, p_f_given_gc, p_c, b, g, c, f)
                        if all(current_evidence[k] == v for k, v in query_variables.items()):
                            total_prob += prob
    return total_prob

def main():
    training_data_file = r"C:\Users\keert\OneDrive\Desktop\Ai3\ass2_k\ass2_k\training_data.txt"
    data = read_training_data(training_data_file)
    counts = count_occurrences(data)
    p_b, p_g_given_b, p_f_given_gc, p_c = calculate_probabilities(counts)

    query_variables, evidence = parse_arguments(['Bt','Ff','Cf'])

    prob = inference_by_enumeration(p_b, p_g_given_b, p_f_given_gc, p_c, query_variables, evidence)
    print(f"P({query_variables} | {evidence}) = {prob}")

main()


P({'B': True, 'G': False, 'F': False} | {}) = 0.0032976332065245226
P({'B': True, 'F': False, 'C': False} | {}) = 0.06952324257642632
